In [1]:
include("../EDKit.jl")
using Main.EDKit, LinearAlgebra, Test, Profile, ITensors, ITensorMPS

In [2]:
function mps2dmpo(ψ::MPS)
    s = siteinds(ψ)
    L = length(s)
    psi = MPO(L)
    
    psi[1] = begin
        l1 = linkind(ψ, 1)
        Cl = combiner(l1, l1', tags="Link,l=1")
        ψ[1]' * conj(ψ[1]) * Cl
    end
    
    for i in 2:L-1 
        li = linkind(ψ, i)
        Cl2 = combiner(li, li', tags="Link,l=$i")
        psi[i] = ψ[i]' * conj(ψ[i]) * Cl * Cl2 
        Cl = Cl2
    end

    psi[L] = begin
        ψ[L]' * conj(ψ[L])* Cl
    end
    psi
end

mps2dmpo (generic function with 1 method)

In [3]:
L = 5
s = siteinds("S=1/2", L)
ps = siteinds("Pauli", L)
vec = rand(ComplexF64, 2^L) |> normalize!
ψ = vec2mps(vec, s)
ρ = vec * vec'
pmps = mps2pmps(ψ, ps)
mpo = pmps2mpo(pmps, s)
inner(ψ', mpo, ψ)

0.2081062977226124 + 8.239936510889834e-18im

In [146]:
t=ITensor(
    [1 0 0 0
     0 1/sqrt(2) 0 1im/sqrt(2)
     0 1/sqrt(2) 0 -1im/sqrt(2)
     0 0 1 0], 
    l1, Index(4))
(pmps[1]*t).tensor[:]

Dim 1: 16
NDTensors.Dense{ComplexF64, SubArray{ComplexF64, 1, Vector{ComplexF64}, Tuple{Base.Slice{Base.OneTo{Int64}}}, true}}
 16-element
      0.5000000000000002 + 0.0im
      0.4751012813103634 + 0.0im
     0.15390559907085527 + 0.0im
    0.024327742844174327 + 0.0im
 -1.1775693440128312e-16 + 0.0im
      0.1639173177404265 + 5.187702921235698e-18im
      -0.419888400903787 + 1.7376904038113167e-17im
     -0.5448163394495843 - 1.0558719559927019e-17im
      0.4999999999999998 + 0.0im
    -0.47510128131036305 + 0.0im
    -0.15390559907085505 + 0.0im
    -0.02432774284417416 + 0.0im
   7.850462293418875e-17 + 0.0im
     0.14727069613227958 + 1.0144209318156e-18im
     -0.5256613586100304 + 1.2224587518748405e-17im
      0.4494346205233535 - 1.7152371496660692e-17im

In [149]:
B1 = ParityBasis(L=2, p=1, base=2)
B2 = ParityBasis(L=2, p=-1, base=2)
B0 = TensorBasis(L=2)
b=DoubleBasis(B1, B0)

DoubleBasis{ParityBasis{Int64}, TensorBasis}([0, 0], ParityBasis{Int64}([0, 0], [1, 2, 4], [2.0, 1.4142135623730951, 2.0], 1, 2), TensorBasis([0, 0], 2), 2)

In [150]:
b=DoubleBasis(B1, B0)
v1 = operator([I(2)], [[1]], b) |> Array
b=DoubleBasis(B2, B0)
v2 = operator([I(2)], [[1]], b) |> Array
vcat(v1, 1im*v2)

4×4 Matrix{ComplexF64}:
 2.0+0.0im      0.0+0.0im          0.0+0.0im      0.0+0.0im
 0.0+0.0im  1.41421+0.0im      1.41421+0.0im      0.0+0.0im
 0.0+0.0im      0.0+0.0im          0.0+0.0im      2.0+0.0im
 0.0+0.0im      0.0+1.41421im     -0.0-1.41421im  0.0+0.0im

In [3]:
function umat(n)
    B1 = ParityBasis(L=2, p=1, base=n)
    B2 = ParityBasis(L=2, p=-1, base=n)
    B0 = TensorBasis(L=2, base=n)
    b=DoubleBasis(B1, B0)
    v1 = operator([I(n)], [[1]], b) |> Array
    b=DoubleBasis(B2, B0)
    v2 = operator([I(n)], [[1]], b) |> Array
    out = Matrix{ComplexF64}(undef, n^2, n^2)
    out[:, 1:size(v1, 1)] = transpose(v1)
    out[:, end-size(v2, 1)+1:end] = 1im*transpose(v2)
    out / 2
end

umat (generic function with 1 method)

In [164]:
n=3
B1 = ParityBasis(L=2, p=1, base=n)
B2 = ParityBasis(L=2, p=-1, base=n)
B0 = TensorBasis(L=2, base=n)
b=DoubleBasis(B1, B0)
v1 = operator([I(n)], [[1]], b) |> Array
b=DoubleBasis(B2, B0)
v2 = operator([I(n)], [[1]], b) |> Array

3×9 Matrix{ComplexF64}:
 0.0+0.0im  1.41421+0.0im      0.0+0.0im  …       0.0+0.0im  0.0+0.0im
 0.0+0.0im      0.0+0.0im  1.41421+0.0im          0.0+0.0im  0.0+0.0im
 0.0+0.0im      0.0+0.0im      0.0+0.0im     -1.41421+0.0im  0.0+0.0im

In [182]:
function mps2pmps2(ψ::MPS, S::AbstractVector)
    s = siteinds(ψ)
    L = length(s)
    psi = MPS(L)
    
    psi[1] = begin
        l1 = linkind(ψ, 1)
        Cl = combiner(l1, l1', tags="Link,l=1")
        C = ITensor(EDKit.PAULI_CONVERSION, S[1], s[1]', s[1])
        ψ[1]' * conj(ψ[1]) * C * Cl
    end
    
    for i in 2:L-1 
        li = linkind(ψ, i)
        Cl2 = combiner(li, li', tags="Link,l=$i")
        C = ITensor(EDKit.PAULI_CONVERSION, S[i], s[i]', s[i])
        psi[i] = ψ[i]' * conj(ψ[i]) * C * Cl * Cl2
        Cl = Cl2
    end

    psi[L] = begin
        C = ITensor(EDKit.PAULI_CONVERSION, S[L], s[L]', s[L])
        ψ[L]' * conj(ψ[L]) * C * Cl
    end
    for i in 1:L-1
        n = linkdim(ψ, i)
        u = umat(n)
        l0 = commonind(psi[i], psi[i+1])
        l = Index(n^2, tags="Link,l=$i")
        U = ITensor(u, l0, l)
        Ud = ITensor(u', l, l0)
        psi[i] = psi[i] * U
        psi[i+1] = Ud * psi[i+1]
    end
    psi
end

mps2pmps2 (generic function with 1 method)

In [196]:
res=mps2pmps2(ψ, ps)

MPS
[1] ((dim=4|id=563|"Pauli,Site,n=1"), (dim=4|id=112|"Link,l=1"))
[2] ((dim=4|id=112|"Link,l=1"), (dim=4|id=939|"Pauli,Site,n=2"), (dim=16|id=1|"Link,l=2"))
[3] ((dim=16|id=1|"Link,l=2"), (dim=4|id=594|"Pauli,Site,n=3"), (dim=16|id=348|"Link,l=3"))
[4] ((dim=16|id=348|"Link,l=3"), (dim=4|id=608|"Pauli,Site,n=4"), (dim=4|id=930|"Link,l=4"))
[5] ((dim=4|id=930|"Link,l=4"), (dim=4|id=618|"Pauli,Site,n=5"))


In [201]:
imag(res[5].tensor[:]) |> norm

3.458818605073972e-18

In [181]:
umat(2)'

4×4 adjoint(::Matrix{ComplexF64}) with eltype ComplexF64:
 1.0-0.0im       0.0-0.0im        0.0-0.0im  0.0-0.0im
 0.0-0.0im  0.707107-0.0im   0.707107-0.0im  0.0-0.0im
 0.0-0.0im       0.0-0.0im        0.0-0.0im  1.0-0.0im
 0.0-0.0im  0.707107-0.0im  -0.707107-0.0im  0.0-0.0im

In [202]:
@time [umat(5) for i in 1:1000];

  0.314681 seconds (494.27 k allocations: 559.321 MiB, 11.53% gc time, 9.73% compilation time)


In [4]:
mat1 = umat(3)

9×9 Matrix{ComplexF64}:
 1.0+0.0im       0.0+0.0im  …   0.0+0.0im        0.0+0.0im
 0.0+0.0im  0.707107+0.0im      0.0+0.0im        0.0+0.0im
 0.0+0.0im       0.0+0.0im      0.0+0.707107im   0.0+0.0im
 0.0+0.0im  0.707107+0.0im      0.0+0.0im        0.0+0.0im
 0.0+0.0im       0.0+0.0im      0.0+0.0im        0.0+0.0im
 0.0+0.0im       0.0+0.0im  …   0.0+0.0im        0.0+0.707107im
 0.0+0.0im       0.0+0.0im     -0.0-0.707107im   0.0+0.0im
 0.0+0.0im       0.0+0.0im      0.0+0.0im       -0.0-0.707107im
 0.0+0.0im       0.0+0.0im      0.0+0.0im        0.0+0.0im

In [5]:
mat2 = EDKit._umat(3)

9×9 Matrix{ComplexF64}:
 1.0+0.0im       0.0+0.0im       0.0+0.0im  …  0.0+0.0im       0.0+0.0im
 0.0+0.0im  0.707107+0.0im       0.0+0.0im     0.0+0.0im       0.0+0.0im
 0.0+0.0im       0.0+0.0im  0.707107+0.0im     0.0+0.707107im  0.0+0.0im
 0.0+0.0im  0.707107+0.0im       0.0+0.0im     0.0+0.0im       0.0+0.0im
 0.0+0.0im       0.0+0.0im       0.0+0.0im     0.0+0.0im       0.0+0.0im
 0.0+0.0im       0.0+0.0im       0.0+0.0im  …  0.0+0.0im       0.0+0.707107im
 0.0+0.0im       0.0+0.0im  0.707107+0.0im     0.0-0.707107im  0.0+0.0im
 0.0+0.0im       0.0+0.0im       0.0+0.0im     0.0+0.0im       0.0-0.707107im
 0.0+0.0im       0.0+0.0im       0.0+0.0im     0.0+0.0im       0.0+0.0im

In [8]:
function _umat(n::Int64)
    B1 = ParityBasis(L=2, p=1, base=n)
    B2 = ParityBasis(L=2, p=-1, base=n)
    n1, n2 = size(B1, 1), size(B2, 1)
    out = zeros(ComplexF64, n^2, n^2)
    for i in 1:n1
        a = 1 / change!(B1, i)
        out[index(B1.dgt; base=n), i] += a 
        reverse!(B1.dgt)
        out[index(B1.dgt; base=n), i] += a 
    end
    for i in 1:n2
        j = n1 + i 
        b = 1im / change!(B2, i) 
        out[index(B2.dgt; base=n), j] += b
        reverse!(B2.dgt)
        out[index(B2.dgt; base=n), j] -= b
    end
    out
end

_umat (generic function with 1 method)

In [7]:
B1 = ParityBasis(L=2, p=1, base=3)
i = 1


ParityBasis{Int64}([0, 0], [1, 2, 3, 5, 6, 9], [2.0, 1.4142135623730951, 1.4142135623730951, 2.0, 1.4142135623730951, 2.0], 1, 3)

In [9]:
_umat(3)

9×9 Matrix{ComplexF64}:
 1.0+0.0im       0.0+0.0im       0.0+0.0im  …  0.0+0.0im       0.0+0.0im
 0.0+0.0im  0.707107+0.0im       0.0+0.0im     0.0+0.0im       0.0+0.0im
 0.0+0.0im       0.0+0.0im  0.707107+0.0im     0.0+0.707107im  0.0+0.0im
 0.0+0.0im  0.707107+0.0im       0.0+0.0im     0.0+0.0im       0.0+0.0im
 0.0+0.0im       0.0+0.0im       0.0+0.0im     0.0+0.0im       0.0+0.0im
 0.0+0.0im       0.0+0.0im       0.0+0.0im  …  0.0+0.0im       0.0+0.707107im
 0.0+0.0im       0.0+0.0im  0.707107+0.0im     0.0-0.707107im  0.0+0.0im
 0.0+0.0im       0.0+0.0im       0.0+0.0im     0.0+0.0im       0.0-0.707107im
 0.0+0.0im       0.0+0.0im       0.0+0.0im     0.0+0.0im       0.0+0.0im

In [12]:
i = 7
[mat2[:,i] mat1[:, i]]

9×2 Matrix{ComplexF64}:
 0.0+0.0im        0.0+0.0im
 0.0+0.707107im   0.0+0.707107im
 0.0+0.0im        0.0+0.0im
 0.0-0.707107im  -0.0-0.707107im
 0.0+0.0im        0.0+0.0im
 0.0+0.0im        0.0+0.0im
 0.0+0.0im        0.0+0.0im
 0.0+0.0im        0.0+0.0im
 0.0+0.0im        0.0+0.0im